# Use case examples
This notebook demonstrates different use-cases for QC Lab

## Running a default simulation

In [1]:
from qc_lab import Simulation
from qc_lab.dynamics import serial_driver, parallel_driver_multiprocessing
from qc_lab.models import SpinBoson
from qc_lab.algorithms import MeanField
import numpy as np

sim_settings = {"num_trajs":100,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.state.wf_db = np.array([1.0, 0.0], dtype=complex)
data = serial_driver(sim)
data = parallel_driver_multiprocessing(sim)

100%|██████████| 10001/10001 [00:02<00:00, 3631.18it/s]


## Changing the model ingredients

In [2]:
from qc_lab import Simulation
from qc_lab.dynamics import serial_driver, parallel_driver_multiprocessing
from qc_lab.models import SpinBoson
from qc_lab.algorithms import MeanField
import numpy as np
from qc_lab.ingredients import harmonic_oscillator_wigner_init_classical

sim_settings = {"num_trajs":100,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.state.wf_db = np.array([1.0, 0.0], dtype=complex)
print(sim.model.ingredients[5])
# change the init_classical ingredient from boltzmann to wigner.
sim.model.ingredients[5] = ('init_classical', harmonic_oscillator_wigner_init_classical)
print(sim.model.ingredients[5])
data = serial_driver(sim)
data = parallel_driver_multiprocessing(sim)

('init_classical', <function harmonic_oscillator_boltzmann_init_classical at 0x763afdd2c8b0>)
('init_classical', <function harmonic_oscillator_wigner_init_classical at 0x763afdd2c940>)


100%|██████████| 10001/10001 [00:03<00:00, 2972.73it/s]


## Changing algorithm tasks

In [3]:
from qc_lab import Simulation
from qc_lab.dynamics import serial_driver, parallel_driver_multiprocessing
from qc_lab.models import SpinBoson
from qc_lab.algorithms import MeanField
import numpy as np

sim_settings = {"num_trajs":100,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.state.wf_db = np.array([1.0, 0.0], dtype=complex)
print(sim.algorithm.output_recipe)
def my_task(algorithm, sim, parameters, state):
    # we can just calculate something simple like the response function
    if sim.t_ind == 0:
        state.wf_db_0 = np.copy(state.wf_db)
    state.response = np.sum(np.conj(state.wf_db_0) * state.wf_db, axis=-1)
    return parameters, state
sim.algorithm.output_recipe.append(my_task)
sim.algorithm.output_variables.append('response')
print(sim.algorithm.output_recipe)
data = serial_driver(sim)
data = parallel_driver_multiprocessing(sim)

[<function update_t at 0x763afdd2eb00>, <function update_dm_db_mf at 0x763afdd2eb90>, <function MeanField._update_quantum_energy at 0x763afdd2fac0>, <function MeanField._update_classical_energy at 0x763afdd2fb50>]
[<function update_t at 0x763afdd2eb00>, <function update_dm_db_mf at 0x763afdd2eb90>, <function MeanField._update_quantum_energy at 0x763afdd2fac0>, <function MeanField._update_classical_energy at 0x763afdd2fb50>, <function my_task at 0x763acef70940>]


100%|██████████| 10001/10001 [00:03<00:00, 3034.33it/s]


## User supplied seeds



In [4]:
np.random.seed(1234)

my_seeds = np.unique(np.random.randint(0,10000, size=100))

from qc_lab import Simulation
from qc_lab.dynamics import serial_driver, parallel_driver_multiprocessing
from qc_lab.models import SpinBoson
from qc_lab.algorithms import MeanField
import numpy as np

sim_settings = {"num_trajs":100,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.state.wf_db = np.array([1.0, 0.0], dtype=complex)
data_serial = serial_driver(sim, seeds=my_seeds)

assert np.all(data_serial.data_dict['seed'] == my_seeds)
print('validated serial_driver with seeds')

data_parallel = parallel_driver_multiprocessing(sim, seeds=my_seeds)

assert np.all(data_parallel.data_dict['seed'] == my_seeds)
print('validated parallel_driver_multiprocessing with seeds')

for key, val in data_serial.data_dict.items():
    if key in data_parallel.data_dict:
        assert np.all(val == data_parallel.data_dict[key]), f"Mismatch in {key} between serial and parallel drivers"
print('validated data consistency between serial and parallel drivers')


/home/akrotz/Research/qc_lab/qc_lab/dynamics/serial_driver.py:29: UserWarning: Setting sim.settings.num_trajs to the number of provided seeds.
  warnings.warn(
100%|██████████| 10001/10001 [00:02<00:00, 4540.57it/s]

validated serial_driver with seeds



/home/akrotz/Research/qc_lab/qc_lab/dynamics/parallel_driver_multiprocessing.py:49: UserWarning: Setting sim.settings.num_trajs to the number of provided seeds.
  warnings.warn(
100%|██████████| 10001/10001 [00:03<00:00, 3222.16it/s]


validated parallel_driver_multiprocessing with seeds
validated data consistency between serial and parallel drivers


## User supplied data object

In [5]:
## Serial
from qc_lab import Simulation
from qc_lab.dynamics import serial_driver, parallel_driver_multiprocessing
from qc_lab.models import SpinBoson
from qc_lab.algorithms import MeanField
import numpy as np

sim_settings = {"num_trajs":100,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.state.wf_db = np.array([1.0, 0.0], dtype=complex)
data_1 = serial_driver(sim)
data_2 = serial_driver(sim, data=data_1)

sim_settings = {"num_trajs":200,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.state.wf_db = np.array([1.0, 0.0], dtype=complex)
data_3 = serial_driver(sim)

for key, val in data_3.data_dict.items():
    if key in data_2.data_dict:
        assert np.all(val == data_2.data_dict[key]), f"Mismatch in {key}"
print('validated data consistency between supplied data and new simulation')

100%|██████████| 10001/10001 [00:01<00:00, 5021.35it/s]


200


100%|██████████| 10001/10001 [00:01<00:00, 5061.35it/s]

validated data consistency between supplied data and new simulation


In [1]:
## Parallel
from qc_lab import Simulation
from qc_lab.dynamics import serial_driver, parallel_driver_multiprocessing
from qc_lab.models import SpinBoson
from qc_lab.algorithms import MeanField
import numpy as np

sim_settings = {"num_trajs":100,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.state.wf_db = np.array([1.0, 0.0], dtype=complex)
data_1 = parallel_driver_multiprocessing(sim)
data_2 = parallel_driver_multiprocessing(sim, data=data_1)
print(len(data_2.data_dict['seed']))
sim_settings = {"num_trajs":200,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.state.wf_db = np.array([1.0, 0.0], dtype=complex)
data_3 = parallel_driver_multiprocessing(sim)

for key, val in data_3.data_dict.items():
    if key in data_2.data_dict:
        print(f"Validating {key}")
        assert np.all(val == data_2.data_dict[key]), f"Mismatch in {key}"
print('validated data consistency between supplied data and new simulation')

100%|██████████| 10001/10001 [00:04<00:00, 2181.98it/s]


200


100%|██████████| 10001/10001 [00:08<00:00, 1205.29it/s]


Validating seed
Validating norm_factor
Validating t
Validating dm_db
Validating classical_energy
Validating quantum_energy
validated data consistency between supplied data and new simulation
